In [71]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torchmetrics.classification import Accuracy
import torchmetrics

import torchvision.models as models

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install torchmetrics

In [5]:
import zipfile
with zipfile.ZipFile("tomato_data.zip", 'r') as zip:
  zip.extractall("tomato_data")

In [6]:
path= '/content/tomato_data'

In [131]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  #Removed RandomCrop(32) as it resized the image back
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])   #pretrained ImageNet values
])

test_transform= transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [132]:
train_path= '/content/tomato_data/data/Training_set'
test_path= '/content/tomato_data/data/Testing_set'

train_data= datasets.ImageFolder(root=train_path, transform= train_transform)
test_set= datasets.ImageFolder(root=test_path, transform= test_transform)

for (img, label) in train_data:
  print(img.shape)
  break

torch.Size([3, 128, 128])


In [133]:
train_size = int(0.85 * len(train_data))
val_size   = len(train_data) - train_size

train_set, val_set = random_split(train_data, [train_size, val_size], generator=torch.Generator().manual_seed(42))


In [153]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_set,  batch_size=64, shuffle=False, num_workers=2)


for (img, label) in train_loader:
  print(img.shape)
  break

torch.Size([64, 3, 128, 128])


In [154]:
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 10)
)
model = model.to(device)

In [155]:

loss_function= nn.CrossEntropyLoss()

In [156]:
optimizer= torch.optim.Adam(model.parameters(), lr= 0.0001)

scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= "max", factor= 0.5, patience= 3, verbose= True)


In [157]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [158]:
val_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            val_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = val_accuracy.compute() * 100  # Convert to %

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

    return accuracy


In [159]:
test_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    test_accuracy.reset()   #Reset metric state for each epoch

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            test_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = test_accuracy.compute() * 100  # Convert to %

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return accuracy

In [160]:
epochs = 50
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_acc = validate(val_loader, model, loss_function)
    scheduler.step(val_acc)
print("\n")
print("-----------------------------------------------------------------")
print("\n")
test(test_loader, model, loss_function)


Epoch 1/50
Training Average Loss: 1.5350
Validation Loss: 1.4801, Validation Accuracy: 50.20%

Epoch 2/50
Training Average Loss: 1.1721
Validation Loss: 1.2772, Validation Accuracy: 51.63%

Epoch 3/50
Training Average Loss: 1.0457
Validation Loss: 1.1343, Validation Accuracy: 57.74%

Epoch 4/50
Training Average Loss: 0.8864
Validation Loss: 1.1271, Validation Accuracy: 60.21%

Epoch 5/50
Training Average Loss: 0.7965
Validation Loss: 0.9812, Validation Accuracy: 59.69%

Epoch 6/50
Training Average Loss: 0.7465
Validation Loss: 1.1364, Validation Accuracy: 56.18%

Epoch 7/50
Training Average Loss: 0.6617
Validation Loss: 0.9769, Validation Accuracy: 65.67%

Epoch 8/50
Training Average Loss: 0.6249
Validation Loss: 0.9833, Validation Accuracy: 61.90%

Epoch 9/50
Training Average Loss: 0.5607
Validation Loss: 0.9105, Validation Accuracy: 67.49%

Epoch 10/50
Training Average Loss: 0.4969
Validation Loss: 0.9678, Validation Accuracy: 68.92%

Epoch 11/50
Training Average Loss: 0.4555
Valida

tensor(71.6863, device='cuda:0')